<a href="https://colab.research.google.com/github/peteray-dev/Text-generation-using-gemma/blob/master/Text_generation_using_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation of datas Using Gemma (Google)

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


In [2]:
!kaggle competitions download llm-prompt-recovery


  0% 0.00/1.45k [00:00<?, ?B/s]
100% 1.45k/1.45k [00:00<00:00, 1.74MB/s]


In [3]:

!mkdir llm-prompt-recovery

!mv /content/llm-prompt-recovery.zip* /content/llm-prompt-recovery


In [4]:
! unzip "/content/llm-prompt-recovery/llm-prompt-recovery.zip"

Archive:  /content/llm-prompt-recovery/llm-prompt-recovery.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [6]:
train_df

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [7]:
train_df['rewrite_prompt'][0]

'Convert this into a sea shanty: """The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."""'

## Generating another text dataset

In [8]:
!kaggle datasets download nelgiriyewithana/emotions

! unzip "emotions.zip"


 32% 5.00M/15.7M [00:00<00:00, 41.9MB/s]
100% 15.7M/15.7M [00:00<00:00, 84.8MB/s]
Archive:  emotions.zip
  inflating: text.csv                


In [9]:
emotion_df = pd.read_csv('/content/text.csv')
# emotion_df = pd.read_csv('/content/llm-prompt-recovery/test.csv')
emotion_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [10]:
emotion_df.set_index('Unnamed: 0', inplace=True)


In [11]:
emotion_df.head()

,text,label
Unnamed: 0,,
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [12]:
emotion_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416809 entries, 0 to 416808
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    416809 non-null  object
 1   label   416809 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [13]:
original_text = emotion_df['text'][:5]

original_text

Unnamed: 0
0        i just feel really helpless and heavy hearted
1    ive enjoyed being able to slouch about relax a...
2    i gave up my internship with the dmrg and am f...
3                           i dont know i feel so lost
4    i am a kindergarten teacher and i am thoroughl...
Name: text, dtype: object

In [14]:
rewrite_prompts = [
    'Explain this to me like I\'m five.',
    'Convert this into a sea shanty.',
    'Make this rhyme.',
]

# Let's generate the written the using gemma

In [15]:
!pip install -q -U immutabledict sentencepiece
# cloning gemma model
!git clone https://github.com/google/gemma_pytorch.git

#Moving the model into gemma model file
!mkdir gemma_model
!mv /content/gemma_pytorch/gemma* /content/gemma_model


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 139 (delta 42), reused 40 (delta 24), pack-reused 68
Receiving objects: 100% (139/139), 2.15 MiB | 17.92 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [18]:
import sys
sys.path.append("/content/gemma_model")
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch


In [22]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/gemma/pyTorch/7b-it-quant")

print("Path to model files:", path)

100%|██████████| 6.79G/6.79G [01:36<00:00, 75.3MB/s]
Extracting model files...


Path to model files: /root/.cache/kagglehub/models/google/gemma/pyTorch/7b-it-quant/2


In [25]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
    print("GPU Device Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

GPU is not available


In [24]:
# model loading

weights_dir = '/root/.cache/kagglehub/models/google/gemma/pyTorch/7b-it-quant/2'
# variant = '7B-it-quant'
# device_type = 'cuda'

# Load the model
VARIANT = "7b-it-quant"
MACHINE_TYPE = "cuda"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# weights_dir = '/kaggle/input/gemma/pytorch/7b-it-quant/2'

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

# Model Config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")
model_config.quant = "quant" in VARIANT

# Model.
device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx